In [1]:
import torch
import numpy as np
import datasets
import os
import pandas as pd
import random
#import umap
import sys
import evaluate
import json
import seaborn as sns
import re
import nltk
from pathlib import Path
from itertools import product
from IPython.core.debugger import set_trace
from datasets import Dataset, DatasetDict, concatenate_datasets
from torch import nn
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Transformer, Pooling
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from IPython.core.debugger import Pdb, set_trace
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import util
#from tqdm.notebook import tqdm
from tqdm import tqdm
from numpy.lib.stride_tricks import sliding_window_view
from pprint import pprint
from scipy.cluster.hierarchy import linkage

from nbtools.utils import files, strings
from nbtools.sent_encoders import from_hf

datasets.disable_caching()

cache_dir = '/data/john/cache'
proot = files.project_root()

# Set this to whatever you want
seed = 10

torch.manual_seed(seed)
np.random.seed(seed)

%load_ext autoreload
%autoreload 2
%matplotlib inline

/data/john/projects/nb/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 10:41:15.810567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-18 10:41:16.518171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load Data

In [2]:
# read the main dataset
fname = (
    '/data/john/projects/llm_eval/data/results/20240502-185939/'
    'annotation_samples_complete.xlsx'
)
ds = Dataset.from_pandas(pd.read_excel(fname))
print(ds)

# Load in reference Data
refs = {
    'all_sides': Dataset.from_json("/data/john/projects/llm_eval/data/all_sides/test.json"),
    'privacy_policy': Dataset.from_csv("/data/john/projects/llm_eval/data/privacy_policy/3p_data.csv")
}
print(refs)

Dataset({
    features: ['Column1', 'doc1', 'doc2', 'response', 'id', 'template', 'dataset', 'system', 'prompt', 'index', 'semf1-use', 'semf1-distil', 'semf1-rob', 'bertscore', 'hashcode', 'model', 'sys_text_id', 'prompt_text_id', 'rougeLsum', 'rougeL', 'rouge1', 'rouge2', 'annotator_1', 'annotator_2', 'annotator_3', 'annotator_agreement'],
    num_rows: 630
})
{'all_sides': Dataset({
    features: ['Key', 'Left', 'Right', 'Ahmed_Intersection', 'Naman_Intersection', 'Helen_Intersection', 'AllSides_Intersection'],
    num_rows: 137
}), 'privacy_policy': Dataset({
    features: ['Company_1', 'Company_2', 'Category', 'Company_Name_1', 'Company_Name_2', 'Company_Len_1', 'Company_Len_2', 'Annotator1', 'Annotator2', 'Annotator3', 'Unnamed: 10'],
    num_rows: 135
})}


In [3]:
ref_cols = {
    'all_sides': [
        "Ahmed_Intersection", 
        "Naman_Intersection", 
        "Helen_Intersection", 
        "AllSides_Intersection"],
    'privacy_policy': [
        "Annotator1",
        "Annotator2",
        "Annotator3"]
}
single_ref_col = [
    refs[sample['dataset']][sample['id']][ref_cols[sample['dataset']][0]]
    for sample in ds
]

tds = ds.add_column('ref', single_ref_col)
print(tds)

Dataset({
    features: ['Column1', 'doc1', 'doc2', 'response', 'id', 'template', 'dataset', 'system', 'prompt', 'index', 'semf1-use', 'semf1-distil', 'semf1-rob', 'bertscore', 'hashcode', 'model', 'sys_text_id', 'prompt_text_id', 'rougeLsum', 'rougeL', 'rouge1', 'rouge2', 'annotator_1', 'annotator_2', 'annotator_3', 'annotator_agreement', 'ref'],
    num_rows: 630
})


In [9]:
from evaluate import load
metric = load('nbansal/semf1')
pred = [d if d else 'test' for d in tds['response']]
ref = [d if d else 'test' for d in tds['ref']]
results = metric.compute(
    predictions=pred, 
    references=ref,
    tokenize_sentences=True,
    multi_references=False,
    verbose=True,
    gpu=0,
)
pprint(results)

Using devices: 0


Some weights of the model checkpoint at sentence-transformers/use-cmlm-multilingual were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Batches: 100%|██████████| 237/237 [00:07<00:00, 32.84it/s]


[Scores(precision=0.27884677052497864, recall=[0.3730679452419281]),
 Scores(precision=0.1617618203163147, recall=[0.1617618203163147]),
 Scores(precision=0.25768429040908813, recall=[0.5715352296829224]),
 Scores(precision=0.007661432959139347, recall=[0.007661432959139347]),
 Scores(precision=0.18708866834640503, recall=[0.5102405548095703]),
 Scores(precision=0.18524643778800964, recall=[0.2981426417827606]),
 Scores(precision=0.10893621295690536, recall=[0.20007652044296265]),
 Scores(precision=0.23671971261501312, recall=[0.36389416456222534]),
 Scores(precision=0.15913578867912292, recall=[0.252993643283844]),
 Scores(precision=0.1363559365272522, recall=[0.5314338207244873]),
 Scores(precision=0.17755424976348877, recall=[0.6328708529472351]),
 Scores(precision=0.18381710350513458, recall=[0.47453030943870544]),
 Scores(precision=0.18902334570884705, recall=[0.4202830195426941]),
 Scores(precision=0.1236676573753357, recall=[0.22755753993988037]),
 Scores(precision=0.40715923905

# Notes

* imo, no need for `multi-references` flag
* should at least give error for empty elements
* breaks when using empty strings
* score class should show f1 also